<img src="data/1_1FMoK_HWvk1IBaMelUXibw.webp" style="height:300px" /> 

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys
Stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Candra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Candra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Implementing helper functions

In [2]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq
def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)
        
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,'',text)
    return text_returned

Finding the set of unique words from all documents of the data set

In [3]:
all_words = []
dict_global = {}
file_folder = 'data/*'
idx = 1
files_with_index = {}
for file in glob.glob(file_folder):
    print(file)
    fname = file
    file = open(file , "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = os.path.basename(fname)
    idx = idx + 1
    
unique_words_all = set(dict_global.keys())

data\1 - Copy (10).txt
data\1 - Copy (2).txt
data\1 - Copy (3).txt
data\1 - Copy (4).txt
data\1 - Copy (5).txt
data\1 - Copy (6).txt
data\1 - Copy (7).txt
data\1 - Copy (8).txt
data\1 - Copy (9).txt
data\1 - Copy.txt
data\1.txt


Defining the linked list

In [4]:
class Node:
    def __init__(self ,docId, freq = None):
        self.freq = freq
        self.doc = docId
        self.nextval = None
    
class SlinkedList:
    def __init__(self ,head = None):
        self.head = head

Making a linkedlist for each word and storing all the nodes (containing the file name and frequency of the respective word ) in the linkedlist.

In [5]:
linked_list_data = {}
for word in unique_words_all:
    linked_list_data[word] = SlinkedList()
    linked_list_data[word].head = Node(1,Node)
word_freq_in_doc = {}
idx = 1
for file in glob.glob(file_folder):
    file = open(file, "r")
    text = file.read()
    text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in Stopwords]
    word_freq_in_doc = finding_all_unique_words_and_freq(words)
    for word in word_freq_in_doc.keys():
        linked_list = linked_list_data[word].head
        while linked_list.nextval is not None:
            linked_list = linked_list.nextval
        linked_list.nextval = Node(idx ,word_freq_in_doc[word])
    idx = idx + 1

Query processing and output generation

In [6]:
query = input('Enter your query:')
query = word_tokenize(query)
connecting_words = []
cnt = 1
different_words = []
bitwise_op = []
for word in query:
    if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
        different_words.append(word.lower())
    else:
        connecting_words.append(word.lower())
print(connecting_words)
total_files = len(files_with_index)
zeroes_and_ones = []
zeroes_and_ones_of_all_words = []
for word in (different_words):
    if word.lower() in unique_words_all:
        zeroes_and_ones = [0] * total_files
        linkedlist = linked_list_data[word].head
        print(word)
        while linkedlist.nextval is not None:
            zeroes_and_ones[linkedlist.nextval.doc - 1] = 1
            linkedlist = linkedlist.nextval
        zeroes_and_ones_of_all_words.append(zeroes_and_ones)
    else:
        print(word," not found")
        sys.exit()
print(zeroes_and_ones_of_all_words)
for word in connecting_words:
    word_list1 = zeroes_and_ones_of_all_words[0]
    word_list2 = zeroes_and_ones_of_all_words[1]
    if word == "and":
        bitwise_op = [w1 & w2 for (w1,w2) in zip(word_list1,word_list2)]
        zeroes_and_ones_of_all_words.remove(word_list1)
        zeroes_and_ones_of_all_words.remove(word_list2)
        zeroes_and_ones_of_all_words.insert(0, bitwise_op);
    elif word == "or":
        bitwise_op = [w1 | w2 for (w1,w2) in zip(word_list1,word_list2)]
        zeroes_and_ones_of_all_words.remove(word_list1)
        zeroes_and_ones_of_all_words.remove(word_list2)
        zeroes_and_ones_of_all_words.insert(0, bitwise_op);
    elif word == "not":
        bitwise_op = [not w1 for w1 in word_list2]
        bitwise_op = [int(b == True) for b in bitwise_op]
        zeroes_and_ones_of_all_words.remove(word_list2)
        zeroes_and_ones_of_all_words.remove(word_list1)
        bitwise_op = [w1 & w2 for (w1,w2) in zip(word_list1,bitwise_op)]
zeroes_and_ones_of_all_words.insert(0, bitwise_op);
        
files = []    
# print(zeroes_and_ones_of_all_words)
lis = zeroes_and_ones_of_all_words[0]
cnt = 1
for index in lis:
    if index == 1:
        files.append(files_with_index[cnt])
    cnt = cnt+1
    
print(files)

[]
sejuk
[[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1]]
[]


In [8]:
# Diasumsikan 'linked_list_data' dan 'unique_words_all' sudah ada.

# Ambil 5 kata unik pertama (misalnya)
unique_words_list = sorted(list(unique_words_all))
sample_words = unique_words_list[:200]

for word in sample_words:
    print(f"\nKata: '{word}'")

    current_node = linked_list_data[word].head
    index_entries = []

    # Lewati node kepala (head dummy)
    if current_node.nextval is not None:
        current_node = current_node.nextval

    # Kumpulkan semua entri index (Doc ID dan Frekuensi)
    while current_node is not None:
        index_entries.append(f"({current_node.doc}, Freq: {current_node.freq})")
        current_node = current_node.nextval

    print(f"   -> Postings List (Doc ID, Freq): {', '.join(index_entries)}")


Kata: 'acara'
   -> Postings List (Doc ID, Freq): (4, Freq: 1), (11, Freq: 1)

Kata: 'ada'
   -> Postings List (Doc ID, Freq): (1, Freq: 1), (4, Freq: 5), (5, Freq: 1), (7, Freq: 4), (8, Freq: 1), (9, Freq: 1), (11, Freq: 1)

Kata: 'adem'
   -> Postings List (Doc ID, Freq): (5, Freq: 1), (7, Freq: 1), (10, Freq: 1)

Kata: 'agak'
   -> Postings List (Doc ID, Freq): (7, Freq: 1), (10, Freq: 1), (11, Freq: 1)

Kata: 'agar'
   -> Postings List (Doc ID, Freq): (4, Freq: 1)

Kata: 'air'
   -> Postings List (Doc ID, Freq): (4, Freq: 1), (6, Freq: 1)

Kata: 'aja'
   -> Postings List (Doc ID, Freq): (8, Freq: 1), (9, Freq: 1)

Kata: 'akses'
   -> Postings List (Doc ID, Freq): (1, Freq: 1), (11, Freq: 1)

Kata: 'aktif'
   -> Postings List (Doc ID, Freq): (8, Freq: 1)

Kata: 'alam'
   -> Postings List (Doc ID, Freq): (4, Freq: 5)

Kata: 'alat'
   -> Postings List (Doc ID, Freq): (3, Freq: 1)

Kata: 'ambruk'
   -> Postings List (Doc ID, Freq): (2, Freq: 1)

Kata: 'anak'
   -> Postings List (Doc I